# Arabic Sentiment Analysis: Model Testing
This notebook is specifically designed to test the **MARBERT** model trained for Arabic sentiment classification.

### Project Info:
- **Model:** UBC-NLP/MARBERT
- **Classes:** 0 (Negative), 1 (Positive), 2 (Neutral)
- **Utilities:** Uses `./utils/preprocessing.py` for consistent text cleaning.

In [22]:
# 0. Setup Environment
# Run this cell if you haven't installed dependencies or if you get 'BertForSequenceClassification' errors
!pip install -q transformers[torch] accelerate datasets evaluate scikit-learn
print("✓ Environment initialized. If you just installed 'accelerate', please restart your kernel.")

✓ Environment initialized. If you just installed 'accelerate', please restart your kernel.


In [23]:
# 1. Import Dependencies
import torch
import json
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification
from utils.preprocessing import clean_arabic_text

print("✓ Dependencies imported")


✓ Dependencies imported


In [24]:
# 2. Load Model & label Map
MODEL_PATH = './final_model'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Loading model from '{MODEL_PATH}' to {device}...")

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    try:
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
    except Exception:
        model = BertForSequenceClassification.from_pretrained(MODEL_PATH)
    model.to(device)
    model.eval()
    
    # Load label mapping
    label_map_path = os.path.join(MODEL_PATH, 'label_map.json')
    if os.path.exists(label_map_path):
        with open(label_map_path, 'r', encoding='utf-8') as f:
            label_dict = json.load(f)
            id2label = {int(k): v for k, v in label_dict['id2label'].items()}
    else:
        id2label = model.config.id2label
        
    print("✓ Model and Tokenizer loaded successfully")
    print(f"  Class labels: {id2label}")
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n--- Troubleshooting Tips ---")
    print("1. Ensure 'final_model' folder exists and contains 'model.safetensors'.")
    print("2. If you see 'BertForSequenceClassification' error, run Cell 0 and Restart Kernel.")
    model = None


Loading model from './final_model' to cpu...


Loading weights: 100%|██████████| 201/201 [00:00<00:00, 427.03it/s, Materializing param=classifier.weight]                                      


✓ Model and Tokenizer loaded successfully
  Class labels: {0: 'negative', 1: 'positive', 2: 'neutral'}


In [25]:
# 3. Unified Prediction Logic
def get_sentiment(text):
    if model is None:
        return "Model not loaded", 0.0
        
    # 1. Clean
    cleaned_text = clean_arabic_text(text)
    
    # 2. Tokenize
    inputs = tokenizer(cleaned_text, return_tensors="pt", 
                       truncation=True, padding=True, max_length=512).to(device)
    
    # 3. Predict
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
    # 4. Result
    idx = torch.argmax(probs, dim=-1).item()
    return id2label[idx], probs[0][idx].item()

print("✓ Prediction function ready")

✓ Prediction function ready


In [26]:
# 4. Run Benchmarks
benchmarks = [
    ("خدمة رائعة وسريعة جداً شكراً لكم", "Positive"),
    ("للأسف تجربة سيئة للغاية ولن أكررها", "Negative"),
    ("المنتج وصل في موعده والتغليف جيد", "Neutral/Positive"),
    ("مستوى عادي جدا كباقي المطاعم", "Neutral")
]

if model:
    print("{:<50} | {:<10} | {:<10}".format("Text", "Prediction", "Confidence"))
    print("-"*75)
    for text, expected in benchmarks:
        label, conf = get_sentiment(text)
        print("{:<50} | {:<10} | {:<10.2%}".format(text[:47] + "...", label.upper(), conf))

Text                                               | Prediction | Confidence
---------------------------------------------------------------------------
خدمة رائعة وسريعة جداً شكراً لكم...                | POSITIVE   | 91.05%    
للأسف تجربة سيئة للغاية ولن أكررها...              | NEUTRAL    | 54.15%    
المنتج وصل في موعده والتغليف جيد...                | POSITIVE   | 83.39%    
مستوى عادي جدا كباقي المطاعم...                    | NEUTRAL    | 77.84%    


In [29]:
# 5. Interactive Test
if model:
    user_input = input("Enter Arabic text to analyze: ")
    if user_input.strip():
        label, conf = get_sentiment(user_input)
        print(f"\nInput: {user_input}")
        print(f"Prediction: {label.upper()} ({conf:.2%} confidence)")


Input: وحشه اوي
Prediction: NEGATIVE (60.95% confidence)
